In [28]:
import pickle
from vocab import Vocab

def read_languages(data_type, use_saved_vocab): ## train / dev te

    chunk_size=1024*1024 # 1MB 단위로 처리
    # path = "/hdd/user15/RNN/dataset/"
    # if data_type == "train":
    #     en_path = path + f"processed_train/train_en.txt"
    #     de_path = path + f"processed_train/train_de.txt"
    # elif data_type == "dev":
    #     en_path = path + f"processed_dev/newstest2013.en"
    #     de_path = path + f"processed_dev/newstest2013.de"
    # elif data_type == "test":
    #     en_path = path + f"processed_test/newstest2014-deen-src.en.2"
    #     de_path = path + f"processed_test/newstest2014-deen-ref.de.2"

    path = "/home/user15/RNN/dataset5/"
    if data_type == "train":
        en_path = path + f"train.en"
        de_path = path + f"train.de"
    elif data_type == "dev":
        en_path = path + f"newstest_2013.en"
        de_path = path + f"newstest_2013.de"
    elif data_type == "test":
        en_path = path + f"newstest2014.en"
        de_path = path + f"newstest2014.de"

    # Read and parse the text file
    with open(en_path, 'r', encoding = 'utf-8') as f1, open(de_path,'r', encoding = 'utf-8') as f2:
        #pairs = [[normalize_string(en_line.strip()), normalize_string(de_line.strip())] for en_line, de_line in zip(f1, f2)]
        pairs = [[en_line.strip(), de_line.strip()] for en_line, de_line in zip(f1, f2)]

        if use_saved_vocab == True:
            with open('vocab_if/input_feed_input_vocab.pkl', 'rb') as f1, open('vocab_if/input_feed_output_vocab.pkl', 'rb') as f2:
                input_vocab = pickle.load(f1)
                output_vocab = pickle.load(f2)
        else:
            input_vocab = Vocab('en')
            output_vocab = Vocab('de')
            
    return input_vocab, output_vocab, pairs
    
    
    
# def filter_pair(pair,max_len):
#     is_good_pair = (len(pair[0].split(' ')) <= max_len) and (len(pair[1].split(' ')) <= max_len)
#     return is_good_pair

def filter_pair(pair,max_len,min_len):
    is_good_pair = (len(pair[0].split(' ')) <= max_len) and (len(pair[1].split(' ')) <= max_len)
    is_good_pair2 = (len(pair[0].split(' ')) >= min_len) and (len(pair[1].split(' ')) >= min_len)

    return is_good_pair & is_good_pair2

def filter_pairs(pairs,max_len,min_len):
    tf_list = [filter_pair(pair,max_len,min_len) for pair in pairs]
    return [pair for pair in pairs if filter_pair(pair,max_len,min_len)], tf_list

def prepare_data(data_type, max_len,min_len, use_saved_vocab): ## train / dev / test
    input_vocab, output_vocab, pairs = read_languages(data_type, use_saved_vocab)
    pairs, tf_list = filter_pairs(pairs,max_len,min_len)


    if use_saved_vocab == True:
        pass
    else:
        for pair in pairs:
            input_vocab.index_words(pair[0])
            output_vocab.index_words(pair[1])
            
        if input_vocab.n_words > 50000:
            input_vocab.trim_vocab(50000)
        if output_vocab.n_words > 50000:
            output_vocab.trim_vocab(50000)
        # 저장
        with open('vocab_if/input_feed_input_vocab.pkl', 'wb') as f1, open('vocab_if/input_feed_output_vocab.pkl', 'wb') as f2:
            pickle.dump(input_vocab, f1)  # to_dict 메서드를 사용하여 dict로 변환
            pickle.dump(output_vocab, f2)  # to_dict 메서드를 사용하여 dict로 변환
            
    return input_vocab, output_vocab, pairs, tf_list



In [9]:
import numpy as np

np.exp(1.75)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (203575018.py, line 3)

In [82]:
input_vocab, output_vocab, pairs, tf_list = prepare_data("test",9,7,True)
filter_pair_path = "filter_ref.txt"

with open (filter_pair_path, 'w', encoding= "utf-8") as f:

     for pair,tf in zip(pairs,tf_list):
        f.write(pair[1]+'\n')

In [ ]:

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
import torch.nn as nn
import torch.nn.functional as F

from model import Seq2Seq
from dataset2 import NMT_Dataset, prepare_data, tensor_from_pair

from tqdm import tqdm




import argparse

parser = argparse.ArgumentParser()
args = parser.parse_args("")

## Device ##
args.device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

## Eval ##
args.criterion = torch.nn.CrossEntropyLoss(ignore_index=2)  # Optional ignore padding



## Data sttting parameter ##
args.is_shuffle = True

## Gradient/Optimizer ##
args.max_grad_norm = 5
args.learning_rate_gamma = 0.5

## Path ##
args.best_loss_model_path = 'model_new/best_loss_model_2.pth'


## Model ##
args.hidden_dimension = 1000
args.max_len = 10
args.min_len =0
args.n_layers = 4
args.dropout = 0.0

## Learning ##
args.lr = 1.0
args.epoch = 10
args.batch_size = 128
args.lr_milestone = [6, 7, 8, 9, 10]
args.use_saved_vocab = False

## Options depend on dropout version
if args.dropout > 0.0:
    args.epoch = 12
    args.lr_milestone = [8, 9, 10, 11, 12]

## data prepareation:


input_vocab, output_vocab, test_pairs = prepare_data('test',args.max_len, args.min_len,True)


test_input_tensors, test_output_tensors,test_source_len_tensors = tensor_from_pair(test_pairs, input_vocab, output_vocab,args.max_len, "test")
del test_pairs


test_nmt_dataset = NMT_Dataset(test_input_tensors, test_output_tensors)
test_dataloader = DataLoader(test_nmt_dataset, batch_size = args.batch_size, shuffle = False, drop_last=False)

test_output_tensors.size()

torch.Size([2661, 51])